<a href="https://colab.research.google.com/github/VGODIE/ML_kaggle_competitions/blob/master/ChineseCharRecognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals
try:
  # %tensorflow_version only exists in Colab.
  import tensorflow.compat.v2 as tf
except Exception:
  pass
tf.enable_v2_behavior()

In [0]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [0]:
! pip install kaggle

In [0]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"vgodie","key":"5f216b50c6912d8ac1efdc0406da9f50"}'}

In [0]:
! mkdir -p ~/.kaggle
! cp kaggle.json ~/.kaggle
! chmod 600 ~/.kaggle/kaggle.json

In [0]:
! kaggle competitions download -c chinese-char-recognition-smmo19

 97% 161M/165M [00:06<00:00, 20.5MB/s]
100% 165M/165M [00:06<00:00, 25.0MB/s]
100% 201M/202M [00:09<00:00, 18.2MB/s]
100% 202M/202M [00:09<00:00, 21.5MB/s]
  0% 0.00/965k [00:00<?, ?B/s]
100% 965k/965k [00:00<00:00, 63.7MB/s]
 94% 184M/195M [00:08<00:00, 25.9MB/s]
100% 195M/195M [00:08<00:00, 24.1MB/s]
 91% 169M/185M [00:10<00:01, 10.8MB/s]
100% 185M/185M [00:10<00:00, 18.1MB/s]
100% 113M/113M [00:04<00:00, 18.1MB/s] 



In [0]:
! ls

kaggle.json	   sample_data	 train-1.npy.zip  train-3.npy.zip
random_labels.csv  test.npy.zip  train-2.npy.zip  train-4.npy.zip


In [0]:
! unzip train-1.npy.zip
! unzip train-2.npy.zip
! unzip train-3.npy.zip
! unzip train-4.npy.zip
! unzip test.npy.zip

Archive:  train-1.npy.zip
  inflating: train-1.npy             
Archive:  train-2.npy.zip
  inflating: train-2.npy             
Archive:  train-3.npy.zip
  inflating: train-3.npy             
Archive:  train-4.npy.zip
  inflating: train-4.npy             
Archive:  test.npy.zip
  inflating: test.npy                


In [0]:
#download and concatenate data
X = np.load("train-1.npy", allow_pickle=True)
for i in ["train-2.npy", "train-3.npy", "train-4.npy"]:
  X = np.concatenate((X, np.load(i,allow_pickle=True)))

# Data preprocessing


In [0]:
y = X[:, 1]
X = X[:,0]

In [0]:
#resize images
from PIL import Image

def resize_im(X):
  data = list()
  for elem in X:
      im = np.array(Image.fromarray(elem).resize((80, 80)))
      data.append(im)
  return np.array(data).reshape(len(X), 80, 80, -1)

In [0]:
 X_res = resize_im(X)
 del X

In [0]:
x_tr = X_res/255 - 0.5
del X_res

In [0]:
# convert target to categorical

from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
encoder.fit(y)
y = tf.keras.utils.to_categorical(encoder.transform(y), 1000)

# Defining model

In [0]:
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import Adam

In [0]:
#model = tf.keras.applications.resnet50.ResNet50(weights= None, include_top=True, input_shape= (80,80,1))
model = tf.keras.applications.ResNet101V2(weights=None, include_top=True, input_shape=(80, 80, 1))

In [0]:
#adagrad = Adagrad()
adam = Adam(learning_rate=0.0005, amsgrad=True)

In [0]:
model.compile(optimizer=adam, loss = "categorical_crossentropy", metrics=["accuracy"])

In [0]:
model.summary()

In [0]:
model.fit(x_tr, y, batch_size= 64, epochs = 6, validation_split=0.1)

Train on 299688 samples, validate on 33299 samples
Epoch 1/6
299688/299688 [==============================] - 650s 2ms/sample - loss: 1.5615 - accuracy: 0.6974 - val_loss: 0.2612 - val_accuracy: 0.9268
Epoch 2/6
299688/299688 [==============================] - 631s 2ms/sample - loss: 0.1048 - accuracy: 0.9717 - val_loss: 0.1553 - val_accuracy: 0.9554
Epoch 3/6
299688/299688 [==============================] - 629s 2ms/sample - loss: 0.0413 - accuracy: 0.9890 - val_loss: 0.1022 - val_accuracy: 0.9727
Epoch 4/6
299688/299688 [==============================] - 628s 2ms/sample - loss: 0.0168 - accuracy: 0.9959 - val_loss: 0.1037 - val_accuracy: 0.9744
Epoch 5/6
299688/299688 [==============================] - 631s 2ms/sample - loss: 0.0081 - accuracy: 0.9983 - val_loss: 0.0820 - val_accuracy: 0.9798
Epoch 6/6
299688/299688 [==============================] - 630s 2ms/sample - loss: 0.0036 - accuracy: 0.9994 - val_loss: 0.0696 - val_accuracy: 0.9841


In [0]:
model.evaluate(x_test, y_test, batch_size=64)

83247/1 [===============================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

[0.0989828671460948, 0.9737408]

# Making predictions

In [0]:
del x_tr

In [0]:
test = np.load("test.npy", allow_pickle=True)
test = resize_im(test)


In [0]:
test = test/255 - 0.5

In [0]:
preds = model.predict(test)

In [0]:
preds = np.argmax(preds, axis=1)

In [0]:
preds = encoder.inverse_transform(preds)

In [0]:
Id = np.arange(1, len(preds) + 1)

In [0]:
predictions = pd.DataFrame({"Id":Id, "Category":preds}, index=None)

In [0]:
predictions.to_csv("preds_2.csv", header=["Id", "Category"], index=False)